Install the required dependencies:

In [7]:
%pip install -q cassio datasets langchain openai tiktoken
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [8]:
%pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255315 sha256=8a58f98ab88ebadc376709cb95fc906e3e703b9e60099f972e1cfd91a1da5a4b
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


Import the packages you'll need:

In [9]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import gensim
import pickle5 as pickle

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [10]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [11]:
from PyPDF2 import PdfReader

### Setup

In [13]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('short_story.pdf')

In [14]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [15]:
raw_text

' \nwww.britishcouncil.org/learnenglishkids  \n© British Council, 201 7 The United Kingdom’s international organisation for educational opportunities and cultural relations. We are registered in En gland as a charity.   \nLittle Red Riding Hood   Short story  \nLittle Red Riding Hood lived in a wood with her mother. One \nday Little Red Riding Hood went to visit her granny. She had \na nice cake in her basket.  \n \nOn her way Little Red Riding Hood met a wolf.  \n‘Hello!’ said the wolf. ‘Where are you going?’   \n \n‘I’m going to see my grandmother. She lives in a house behind those trees.’  \n \nThe wolf ran to Granny’s house and ate Granny up. He got into Granny’s bed. A little \nlater, Little Red Riding Hood reached the house. She looked at the wolf.  \n \n‘Granny, what big eyes y ou have!’  \n‘All the better to see you with!’ said the wolf.  \n \n‘Granny, what big ears you have!’  \n‘All the better to hear you with!’ said the wolf.  \n \n‘Granny, what a big nose you have!’  \n‘All

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [16]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = ".",
    chunk_size = 10,
    chunk_overlap  = 5,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [30]:
texts[:50]

['www',
 'britishcouncil',
 'org/learnenglishkids  \n© British Council, 201 7 The United Kingdom’s international organisation for educational opportunities and cultural relations',
 'We are registered in En gland as a charity',
 'Little Red Riding Hood   Short story  \nLittle Red Riding Hood lived in a wood with her mother',
 'One \nday Little Red Riding Hood went to visit her granny',
 'She had \na nice cake in her basket',
 'On her way Little Red Riding Hood met a wolf',
 '‘Hello!’ said the wolf',
 '‘Where are you going?’   \n \n‘I’m going to see my grandmother',
 'She lives in a house behind those trees',
 '’  \n \nThe wolf ran to Granny’s house and ate Granny up',
 'He got into Granny’s bed',
 'A little \nlater, Little Red Riding Hood reached the house',
 'She looked at the wolf',
 '‘Granny, what big eyes y ou have!’  \n‘All the better to see you with!’ said the wolf',
 '‘Granny, what big ears you have!’  \n‘All the better to hear you with!’ said the wolf',
 '‘Granny, what a big no

In [18]:
# Remove '\n' from each chunk of text
cleaned_texts = [chunk.replace('\n', '') for chunk in texts]


In [19]:
cleaned_texts[2]

'org/learnenglishkids  © British Council, 201 7 The United Kingdom’s international organisation for educational opportunities and cultural relations'

In [20]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert words in each chunk into vectors
def words_to_vectors(texts):
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Tokenize each chunk into words and fit TF-IDF Vectorizer
    tfidf_vectorizer.fit(texts)

    # Transform each word into a vector
    word_vectors = tfidf_vectorizer.transform(texts)

    return word_vectors, tfidf_vectorizer



In [21]:
# Store vectors in a file using pickle
def save_vectors_to_file(text_vectors, filename):
    with open(filename, 'wb') as f:
        pickle.dump(text_vectors, f)

# Load vectors from a file
def load_vectors_from_file(filename):
    with open(filename, 'rb') as f:
        text_vectors = pickle.load(f)
    return text_vectors


In [22]:
import gensim
import pickle


word_vectors, tfidf_vectorizer = words_to_vectors(cleaned_texts)



In [23]:
# Example usage to see the importance of every word in a chunk
chunk_index = 10  # Index of the chunk for which you want to see word importance

# Get TF-IDF weights for the specified chunk
tfidf_weights_chunk = word_vectors[chunk_index]

# Get the feature names (words) from the TF-IDF Vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a dictionary to store word importance
word_importance = {}

# Iterate over each word index and its corresponding TF-IDF weight
for word_index, tfidf_weight in zip(tfidf_weights_chunk.indices, tfidf_weights_chunk.data):
    # Get the word corresponding to the index
    word = feature_names[word_index]
    # Store the TF-IDF weight of the word in the dictionary
    word_importance[word] = tfidf_weight

# Print word importance
for word, importance in word_importance.items():
    print(f"Word: {word}, Importance: {importance}")


Word: trees, Importance: 0.41876224069471213
Word: those, Importance: 0.41876224069471213
Word: she, Importance: 0.33899876346111957
Word: lives, Importance: 0.41876224069471213
Word: in, Importance: 0.2923401203523422
Word: house, Importance: 0.3133206591937176
Word: behind, Importance: 0.41876224069471213


In [24]:
def question_to_vector(question, tfidf_vectorizer):
    # Transform the question into a TF-IDF vector
    question_vector = tfidf_vectorizer.transform([question])
    return question_vector


In [37]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the question
# Where did little red riding hood live
# what did red riding hood had in her basket
# where was the wood cutter
# red riding hood whom did she meet on her way
# wher did the woodcutter hit the wolf
# what happened when the wolf opened his mouth
# what did woodcutter do when he heard a loud scream
question = ""

# Convert the question into a vector using TF-IDF
question_vector = question_to_vector(question, tfidf_vectorizer)

# Calculate cosine similarity between question vector and text vectors
similarities = cosine_similarity(question_vector, word_vectors)

# Find the index of the most similar chunk
most_similar_index = np.argmax(similarities)

# Retrieve the most similar chunk
most_similar_chunk = texts[most_similar_index]

print("Most similar chunk to answer the question:", most_similar_chunk)

print("\n\n\n")
# Calculate cosine similarity for the first 4 chunks
# Sort similarity scores and retrieve indices of top 4 chunks
top_indices = np.argsort(similarities, axis=1)[0][-4:][::-1]

# Print the similarity scores and corresponding chunks for the top 4 chunks
for idx in top_indices:
    similarity_score = similarities[0][idx]
    chunk = texts[idx]
    print(f"Similarity score for chunk {idx + 1}: {similarity_score}")
    print(f"Chunk {idx + 1}: {chunk}\n")



Most similar chunk to answer the question: ‘Where are you going?’   
 
‘I’m going to see my grandmother




Similarity score for chunk 10: 0.7155414938921723
Chunk 10: ‘Where are you going?’   
 
‘I’m going to see my grandmother

Similarity score for chunk 9: 0.20036322481096547
Chunk 9: ‘Hello!’ said the wolf

Similarity score for chunk 16: 0.14893659466678966
Chunk 16: ‘Granny, what big eyes y ou have!’  
‘All the better to see you with!’ said the wolf

Similarity score for chunk 18: 0.14357153016191224
Chunk 18: ‘Granny, what a big nose you have!’  
‘All the better to smell you with!’ said the wolf



In [18]:
# Retrieve the TF-IDF value for the word at index 111 in the sparse matrix representation
tfidf_value = word_vectors[0, 111]
print("TF-IDF value for the word at index 111:", tfidf_value)


TF-IDF value for the word at index 111: 1.0
